## `IPerf2BoundPair` example
IPerf is a command line tool that requires operation of two process: a server (listener) and a client (sender). The `IPerf2BoundPair` provides a single point of interaction for configuring and running this pair, and merges the CSV output from the two processes into a single table. The below template instantiates a configured `IPerf2BoundPair` instance with support for all command line flags, which in the current version are a subset of [these IPerf2 command line flags](https://iperf2.sourceforge.io/iperf-manpage.html).

In [1]:
import labbench as lb
from ssmdevices.software import IPerf2BoundPair

lb.show_messages('debug')

iperf = IPerf2BoundPair(
    # in practice, select IP addresses that guarantee routing through the appropriate interfaces
    server='127.0.0.1',
    client='127.0.0.1',

    # as a TODO, it looks like it is possible to specify a specific network interface, but this isn't yet supported
    # by IPerf2BoundPair

    ### the parameters below set the corresponding iperf command line flags
    # tcp_window_size=8196 # -w (default may be OS dependent?) https://en.wikipedia.org/wiki/TCP_window_scale_option
    # buffer_size='16k'    # -l size of the transmit buffer (default unknown? possible strange results for UDP)
    interval=0.25,         # -i reporting interval in seconds (default is no output until the end of process)
    # bidirectional=True,  # -d send data both ways (default is False)
    # udp=False,           # -u (default is False (TCP))
    # bit_rate='1M'        # -b bit rate throttling (default is no bit rate throttling)
    # nodelay=True,        # -N (default is False; TCP only) https://www.extrahop.com/company/blog/2016/tcp-nodelay-nagle-quickack-best-practices/
    # mss=1460,            # -M (default 1460? - TCP only, of course) https://en.wikipedia.org/wiki/Maximum_segment_size

    # controls on the duration of the acquisition before the end of operation.
    # these only apply to blocking operation, in which case you can probably set no more than one of these
    time=4,                # -t (how long to run the iperf client; iperf's default is 10s)
    # number=-1,           # -n (by default, iperf uses -t to determine client test length; set -1 to run until killed)
)

### Approach 1: Blocking operation
Blocking operation with `profile()` or `profile(block=True)` doesn't return until after iperf3 process exits. Profiling data is returned from the function as a `pandas.DataFrame`.

In [2]:
# Approach 1: blocking (pipe).
# Calling pipe() doesn't return until the test is done.

with iperf:
    data = iperf.profile()

data

 DEBUG  2023-06-07 11:01:28.862 • IPerf2BoundPair(''): opened
 DEBUG  2023-06-07 11:01:28.866 • IPerf2(None): opened
 DEBUG  2023-06-07 11:01:28.868 • IPerf2('127.0.0.1'): opened
 DEBUG  2023-06-07 11:01:28.871 • IPerf2(None): background execute: 'C:\\Users\\dkuester\\Documents\\src\\ssmdevices\\ssmdevices\\lib\\iperf.exe -s -p 5201 -i 0.25 -B 127.0.0.1 -y C'
 DEBUG  2023-06-07 11:01:28.882 • IPerf2('127.0.0.1'): shell execute ''C:\\Users\\dkuester\\Documents\\src\\ssmdevices\\ssmdevices\\lib\\iperf.exe -c 127.0.0.1 -p 5201 -i 0.25 -B 127.0.0.1:1261 -t 4.0 -y C''
 DEBUG  2023-06-07 11:01:32.950 • IPerf2('127.0.0.1'): ► 20230607110129,127.0.0.1,1261,127.0.0.1,5201,384,0.0-0.3,674103296,21571305472
► 20230607110129,127.0.0.1,1261,127.0.0.1,5201,384,0.3-0.5,1026555904,32849788928
► 20230607110129,127.0.0.1,1261,127.0.0.1,5201,384,0.5-0.8,888143872,28420603904
► 20230607110129,127.0.0.1,1261,127.0.0.1,5201,384,0.8-1.0,1031143424,32996589568
► 20230607110130,127.0.0.1,1261,127.0.0.1,5201,38

,timestamp,client_source_address,client_source_port,client_destination_address,client_destination_port,client_bits_per_second,server_source_address,server_source_port,server_destination_address,server_destination_port,server_bits_per_second
0,2023-06-07 11:01:29.000,127.0.0.1,1261,127.0.0.1,5201,21571305472,127.0.0.1,5201,127.0.0.1,1261,21571305472
1,2023-06-07 11:01:29.250,127.0.0.1,1261,127.0.0.1,5201,32849788928,127.0.0.1,5201,127.0.0.1,1261,32849788928
2,2023-06-07 11:01:29.500,127.0.0.1,1261,127.0.0.1,5201,28420603904,127.0.0.1,5201,127.0.0.1,1261,28420603904
3,2023-06-07 11:01:29.750,127.0.0.1,1261,127.0.0.1,5201,32996589568,127.0.0.1,5201,127.0.0.1,1261,32996589568
4,2023-06-07 11:01:30.000,127.0.0.1,1261,127.0.0.1,5201,32417775616,127.0.0.1,5201,127.0.0.1,1261,32413581312
5,2023-06-07 11:01:30.250,127.0.0.1,1261,127.0.0.1,5201,33101447168,127.0.0.1,5201,127.0.0.1,1261,33105641472
6,2023-06-07 11:01:30.500,127.0.0.1,1261,127.0.0.1,5201,33915142144,127.0.0.1,5201,127.0.0.1,1261,33915142144
7,2023-06-07 11:01:30.750,127.0.0.1,1261,127.0.0.1,5201,33030144000,127.0.0.1,5201,127.0.0.1,1261,33030144000
8,2023-06-07 11:01:31.000,127.0.0.1,1261,127.0.0.1,5201,32296140800,127.0.0.1,5201,127.0.0.1,1261,32296140800
9,2023-06-07 11:01:31.250,127.0.0.1,1261,127.0.0.1,5201,34313601024,127.0.0.1,5201,127.0.0.1,1261,34313601024


### Approach 2: non-blocking (background) call
`profile(block=False)` returns immediately, leaving iperf client and server processes running in the background.
They continue running until stopped; data can be collected intermediately with the `read_stdout()` method, 
which returns a `pandas.DataFrame`.

In [3]:
with iperf:
    # start running; after this, the profiling will keep running until
    # you exit the with block, or run iperf.kill()
    iperf.profile(block=False)

    # collect data for this interval
    lb.sleep(12)

    # each call to read_stdout() clears data in the buffer
    data = iperf.read_stdout()

    # note above - this is actually unecessary right before leaving the with block
    iperf.kill()

data

 DEBUG  2023-06-07 11:01:33.471 • IPerf2BoundPair(''): opened
 DEBUG  2023-06-07 11:01:33.471 • IPerf2(None): opened
 DEBUG  2023-06-07 11:01:33.471 • IPerf2('127.0.0.1'): opened
 DEBUG  2023-06-07 11:01:33.483 • IPerf2(None): background execute: 'C:\\Users\\dkuester\\Documents\\src\\ssmdevices\\ssmdevices\\lib\\iperf.exe -s -p 5201 -i 0.25 -B 127.0.0.1 -y C'
 DEBUG  2023-06-07 11:01:33.491 • IPerf2('127.0.0.1'): background execute: 'C:\\Users\\dkuester\\Documents\\src\\ssmdevices\\ssmdevices\\lib\\iperf.exe -c 127.0.0.1 -p 5201 -i 0.25 -B 127.0.0.1:1264 -t 0.0 -n -1 -y C'
 DEBUG  2023-06-07 11:01:45.516 • IPerf2('127.0.0.1'): killing process 11392
 DEBUG  2023-06-07 11:01:45.525 • IPerf2('127.0.0.1'): process ended
 DEBUG  2023-06-07 11:01:45.537 • IPerf2(None): killing process 19528
 DEBUG  2023-06-07 11:01:45.551 • IPerf2(None): process ended
 DEBUG  2023-06-07 11:01:45.561 • IPerf2BoundPair(''): closed


,timestamp,client_source_address,client_source_port,client_destination_address,client_destination_port,client_bits_per_second,server_source_address,server_source_port,server_destination_address,server_destination_port,server_bits_per_second
0,2023-06-07 11:01:33.000,127.0.0.1,1264,127.0.0.1,5201,27837595648,127.0.0.1,5201,127.0.0.1,1264,27837595648
1,2023-06-07 11:01:34.250,127.0.0.1,1264,127.0.0.1,5201,33084669952,127.0.0.1,5201,127.0.0.1,1264,33084669952
2,2023-06-07 11:01:34.500,127.0.0.1,1264,127.0.0.1,5201,32400998400,127.0.0.1,5201,127.0.0.1,1264,32400998400
3,2023-06-07 11:01:34.750,127.0.0.1,1264,127.0.0.1,5201,32652656640,127.0.0.1,5201,127.0.0.1,1264,32652656640
4,2023-06-07 11:01:34.000,127.0.0.1,1264,127.0.0.1,5201,32782680064,127.0.0.1,5201,127.0.0.1,1264,32786874368
5,2023-06-07 11:01:35.250,127.0.0.1,1264,127.0.0.1,5201,33969668096,127.0.0.1,5201,127.0.0.1,1264,33969668096
6,2023-06-07 11:01:35.500,127.0.0.1,1264,127.0.0.1,5201,32619102208,127.0.0.1,5201,127.0.0.1,1264,32614907904
7,2023-06-07 11:01:35.750,127.0.0.1,1264,127.0.0.1,5201,33638318080,127.0.0.1,5201,127.0.0.1,1264,33638318080
8,2023-06-07 11:01:35.000,127.0.0.1,1264,127.0.0.1,5201,33676066816,127.0.0.1,5201,127.0.0.1,1264,33676066816
9,2023-06-07 11:01:36.250,127.0.0.1,1264,127.0.0.1,5201,34439430144,127.0.0.1,5201,127.0.0.1,1264,34439430144
